In [1]:
#!pip install opencv-python

In [2]:
import threading
import cv2 as cv
import numpy as np

face_cascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")

In [3]:
# Tegeleb kasutaja poolt saadud sisenditega
def inputWorker(ref):
    filters = ["glasses", "sunglasses", "fedora"]
    while True:
        ref[0] = input("Küsimus")
        if ref[0] == "exit":
            break

        if ref[0] in filters:
            ref[1] = ref[0]

In [4]:
# Paneb filtri pildile
def overlayFilter(image, x, y, w, h, filter_img):
    # Võtame overlay pildist alpha kanali välja
    alpha = filter_img[:,:,3]
    alpha = cv.merge([alpha,alpha,alpha])

    cropped_image = image[y:y + h, x:x + w]

    # Võtame overlay pildist BGR kanalid välja
    bgr = filter_img[:,:,0:3]

    # Kui alpha on 0, vali taust, kui mitte, vali meie overlay filter
    image[y:y + h, x:x + w] = np.where(alpha==0, cropped_image, bgr)
    return image

In [5]:
# Valib vastava filtri, määrab selle suuruse ning paneb selle pildile
def filterImage(image, x, y, w, h, filter_name):
    head_filters = ["fedora"]
    eye_filters = ["glasses", "sunglasses"]

    filter_img = cv.imread(f"filters/{filter_name}.png", cv.IMREAD_UNCHANGED)

    fh = int(filter_img.shape[0] * w / float(filter_img.shape[1]))
    filter_img = cv.resize(filter_img, (w, fh))

    fh, fw = filter_img.shape[:2]
    if filter_name in head_filters:
        y = y - int(fw * 0.5)
    elif filter_name in eye_filters:
        y = y + int(fw * 0.25)

    # Kui filter läheb ekraanist välja, siis lõikame vastavalt vähemaks
    if y < 0:
        filter_img = filter_img[abs(y):]
        h -= abs(y)
        y = 0

    if 0 <= x and x + fw < image.shape[1] and y + fh < image.shape[0]:
        return overlayFilter(image, x, y, fw, fh, filter_img)
    return image

In [6]:
video = cv.VideoCapture(0)
refs = ["", ""]

threading.Thread(target=inputWorker, args=[refs]).start()

while True:
    result, img = video.read()

    # Flipib pildi ümber, et toimuks arusaadav peegeldus
    img = cv.flip(img, 1)

    # Näotuvastus
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_img)
    for (x, y, w, h) in faces:
        if refs[1] != "":
            img = filterImage(img, x, y, w, h, refs[1])

    cv.imshow("Camera", img)

    k = cv.waitKey(30) & 0xFF
    if k == 27: # ehk ESC nupp
        break

    if refs[0] == "exit":
        break

video.release()
cv.destroyAllWindows()